In [112]:
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(URL) 
soup = BeautifulSoup(req.content, 'html5lib') 
table = soup.find('tbody')
li_ad=[]
for row in table.find_all('tr'): 
    neigh = {}
    cell = row.find_all('td')
    if cell:
        neigh['Postal_Code']=cell[0].get_text()
        neigh['Borough']=cell[1].get_text()
        neigh['District']=cell[2].get_text()
        li_ad.append(neigh)
import pandas as pd
import numpy as np
pd.set_option("display.precision", 8)
neighbours = pd.DataFrame(li_ad)
neighbours = neighbours.replace('\n','', regex=True)
neighbours['Postal_Code'] = neighbours['Postal_Code'].replace(' ','', regex=True)
neighbours=neighbours[neighbours.Borough != 'Not assigned']
print(neighbours.shape)
neighbours.head()

(103, 3)


,Postal_Code,Borough,District
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [113]:

neighbours['latitude'] = 'nan'
neighbours['longitude'] = 'nan'
print(neighbours.shape)
neighbours.head()


(103, 5)


,Postal_Code,Borough,District,latitude,longitude
2,M3A,North York,Parkwoods,nan,nan
3,M4A,North York,Victoria Village,nan,nan
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",nan,nan
5,M6A,North York,"Lawrence Manor, Lawrence Heights",nan,nan
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",nan,nan


In [116]:
from geopy.geocoders import Nominatim
#geolocator = Nominatim(user_agent="ny_explorer")
#location = geolocator.geocode({"postalcode": 400065})
#print(location.latitude,location.longitude)
geo_code=pd.read_csv('can_data.csv')

for ind in neighbours.index:    
    # initialize your variable to None
    for jnd in geo_code.index:
        if neighbours['Postal_Code'][ind] == geo_code['Postal Code'][jnd]:
            neighbours['latitude'][ind] = geo_code['Latitude'][jnd]    
            neighbours['longitude'][ind] = geo_code['Longitude'][jnd]




    
    
neighbours.head()


,Postal_Code,Borough,District,latitude,longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [117]:
#.shape
neighbours.dropna(inplace=True)
print(neighbours.shape)

(103, 5)


In [118]:
from geopy.geocoders import Nominatim 
address = 'Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 61.0666922, -107.9917071.


In [166]:
bor=neighbours.groupby('Borough').count().reset_index()
bor=bor[['Borough']]
bor['lati']='nan'
bor['long']='nan'
bor

,Borough,lati,long
0,Central Toronto,nan,nan
1,Downtown Toronto,nan,nan
2,East Toronto,nan,nan
3,East York,nan,nan
4,Etobicoke,nan,nan
5,Mississauga,nan,nan
6,North York,nan,nan
7,Scarborough,nan,nan
8,West Toronto,nan,nan
9,York,nan,nan


In [139]:
neighbours.loc[neighbours['Borough']==bor['Borough'][0]]['latitude']

93      43.72802
94     43.711695
102    43.712751
103    43.696948
111    43.715383
112     43.67271
120    43.704324
129    43.689574
138    43.686412
Name: latitude, dtype: object

In [168]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ca_explorer")

for ind in bor.index:
    
    address = bor['Borough'][ind]+', Canada'


    location = geolocator.geocode(address)
    bor['lati'][ind] = location.latitude
    bor['long'][ind] = location.longitude
bor
    

,Borough,lati,long
0,Central Toronto,43.653482,-79.383935
1,Downtown Toronto,43.656322,-79.380916
2,East Toronto,43.721789,-79.374027
3,East York,43.699971,-79.33252
4,Etobicoke,43.643556,-79.565633
5,Mississauga,43.590338,-79.645729
6,North York,43.754326,-79.449117
7,Scarborough,43.773077,-79.257774
8,West Toronto,43.653482,-79.383935
9,York,46.088526,-66.930803


In [169]:
import folium
map_cd = folium.Map(location=[ bor['lati'][0], bor['long'][0]], zoom_start=10)

# add markers to map
for lat, lng, label in zip(neighbours.loc[neighbours['Borough']==bor['Borough'][0]]['latitude'],
                           neighbours.loc[neighbours['Borough']==bor['Borough'][0]]['longitude'],
                           neighbours.loc[neighbours['Borough']==bor['Borough'][0]]['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cd)  
    
map_cd

In [142]:
map_cd = folium.Map(location=[ bor['long'][0], bor['lati'][0]], zoom_start=8)

# add markers to map
for lat, lng, label in zip(neighbours['latitude'],
                           neighbours['longitude'],
                           neighbours['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cd)  
    
map_cd

In [151]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
quer='Food'
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    bor['lati'][1], 
    bor['long'][1],
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id=KHMROM1TARSXLRC0I31YIV1RU1KYB0MPPTN24X4STFUX125N&client_secret=PF3D05APUR41GGNOBW32302F0WVQA0RSKX24A3TWEB5FXBFY&v=20190909&ll=-79.3809161,43.6563221&radius=2000&limit=10'

In [336]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
ny_la,ny_lo =43.653482,-79.383935#43.6532,-79.3832#40.7151482,-74.0156573
bor['api_url']='nan'
for ind in bor.index:
    url = 'https://api.foursquare.com/v2/venues/explore?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        (bor['lati'][ind]),
        #ny_la, 
        (bor['long'][ind]),
        #ny_lo, 
        radius, 
        LIMIT)
    bor['api_url'][ind]=url
#url # display URL
bor

,Borough,lati,long,api_url
0,Central Toronto,43.653482,-79.383935,https://api.foursquare.com/v2/venues/explore?c...
1,Downtown Toronto,43.656322,-79.380916,https://api.foursquare.com/v2/venues/explore?c...
2,East Toronto,43.721789,-79.374027,https://api.foursquare.com/v2/venues/explore?c...
3,East York,43.699971,-79.33252,https://api.foursquare.com/v2/venues/explore?c...
4,Etobicoke,43.643556,-79.565633,https://api.foursquare.com/v2/venues/explore?c...
5,Mississauga,43.590338,-79.645729,https://api.foursquare.com/v2/venues/explore?c...
6,North York,43.754326,-79.449117,https://api.foursquare.com/v2/venues/explore?c...
7,Scarborough,43.773077,-79.257774,https://api.foursquare.com/v2/venues/explore?c...
8,West Toronto,43.653482,-79.383935,https://api.foursquare.com/v2/venues/explore?c...
9,York,46.088526,-66.930803,https://api.foursquare.com/v2/venues/explore?c...


In [339]:
dict_res ={}
for _ in bor.index:
    results = requests.get(bor['api_url'][_]).json()
    dict_res[bor['Borough'][_]] = results
dict_res['Central Toronto']

{'meta': {'code': 200, 'requestId': '5ed2a19d216785001b83a5d7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.66248170900001,
    'lng': -79.37151886118865},
   'sw': {'lat': 43.64448169099999, 'lng': -79.39635053881135}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537773d1498e74a75bb75c1e',
       'name': 'Eggspectation Bell Trinity Square',
       'location': {'address': '483 Bay Street',
        'crossStreet': 'Albert Street',
        'lat': 43.65314383888587,
        

In [340]:
for neig in dict_res:
    print(neig)

Central Toronto
Downtown Toronto
East Toronto
East York
Etobicoke
Mississauga
North York
Scarborough
West Toronto
York


In [352]:
dict_res['North York']['response']['totalResults']
#df_det.shape

14

In [353]:
from pandas.io.json import json_normalize
import pandas as pd
df_det = pd.DataFrame(columns=v.columns)
for neig in dict_res:
    venue=[]
   
    for _ in range(len(dict_res[neig]['response']['groups'][0]['items'])):
        if dict_res[neig]['response']['totalResults'] > 0:
            venue.append(dict_res[neig]['response']['groups'][0]['items'][_])#['venue']
    if dict_res[neig]['response']['totalResults'] > 0:
        nearby_venues = json_normalize(venue)
        print('Dataframe  created',nearby_venues.shape)

        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        #print('Dataframe  created')

        nearby_venues = nearby_venues[['venue.id','venue.name','venue.location.lat','venue.location.lng','venue.location.distance','venue.location.formattedAddress','venue.categories']]
        nearby_venues.columns = ['id','name','latitude','longitude','distance_from_location','address','cuisine']
        nearby_venues['Borough'] = neig
        df_det= df_det.append(nearby_venues,ignore_index=True)
df_det.shape
# tranform venues into a dataframe


Dataframe  created (50, 22)
Dataframe  created (50, 22)
Dataframe  created (6, 20)
Dataframe  created (8, 21)
Dataframe  created (10, 21)
Dataframe  created (50, 21)
Dataframe  created (14, 22)
Dataframe  created (40, 21)
Dataframe  created (50, 22)


<ipython-input-353-6793f0bc4b1a>:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venue)


(278, 8)

In [296]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [208]:
nearby_venues['venue.location.labeledLatLngs'][0]
for col in nearby_venues.columns: 
    print(col)

referralId
reasons.count
reasons.items
venue.id
venue.name
venue.location.address
venue.location.crossStreet
venue.location.lat
venue.location.lng
venue.location.labeledLatLngs
venue.location.distance
venue.location.cc
venue.location.city
venue.location.state
venue.location.country
venue.location.formattedAddress
venue.categories
venue.photos.count
venue.photos.groups
venue.location.postalCode
venue.location.neighborhood
venue.venuePage.id


In [330]:
import pandas as pd


v= v.append(nearby_venues,ignore_index=True)
v.shape


(113, 8)

In [356]:
df_det

,id,name,latitude,longitude,distance_from_location,address,cuisine,Borough
0,537773d1498e74a75bb75c1e,Eggspectation Bell Trinity Square,43.65314384,-79.38198017,161,"[483 Bay Street (Albert Street), Toronto ON M5...",Breakfast Spot,Central Toronto
1,4ae7b27df964a52068ad21e3,Japango,43.65526772,-79.38516507,222,"[122 Elizabeth St. (at Dundas St. W), Toronto ...",Sushi Restaurant,Central Toronto
2,4e5d8181a8092f63968617ee,Crepe Delicious,43.65453649,-79.38088886,271,"[220 Yonge St., Toronto ON M5B 2H1, Canada]",Fast Food Restaurant,Central Toronto
3,57bcd3b7498e652a678d0378,Poke Guys,43.65489528,-79.38505238,181,"[112 Elizabeth St (at Dundas St W), Toronto ON...",Poke Place,Central Toronto
4,59246b5aad1789316b35d66c,JOEY Eaton Centre,43.65540380,-79.38192890,268,"[1 Dundas St W, Toronto ON M5G 1Z3, Canada]",Restaurant,Central Toronto
...,...,...,...,...,...,...,...,...
273,4b744336f964a520d8d02de3,Somethin' 2 Talk About,43.65839479,-79.38533766,558,"[78 Gerrard St W, Toronto ON M5G 1J5, Canada]",Middle Eastern Restaurant,West Toronto
274,4ad4c05df964a52059f620e3,Canoe,43.64745207,-79.38132002,703,"[66 Wellington St West (at Bay Street), Toront...",Restaurant,West Toronto
275,4b114234f964a520637923e3,Pumpernickel's Deli,43.64883150,-79.38197006,541,[100 King St. W (at First Canadian Place Food ...,Deli / Bodega,West Toronto
276,506a3830e4b0677a1f9d8eda,Ninki Izakaya,43.65022782,-79.38486260,369,[133 Richmond Street West (Richmond And Univer...,Japanese Restaurant,West Toronto


In [357]:
df_det.groupby('cuisine').count()

,id,name,latitude,longitude,distance_from_location,address,Borough
cuisine,,,,,,,
American Restaurant,13,13,13,13,13,13,13
Asian Restaurant,8,8,8,8,8,8,8
Bakery,10,10,10,10,10,10,10
Bistro,1,1,1,1,1,1,1
Breakfast Spot,10,10,10,10,10,10,10
Burger Joint,2,2,2,2,2,2,2
Burrito Place,3,3,3,3,3,3,3
Café,23,23,23,23,23,23,23
Chinese Restaurant,11,11,11,11,11,11,11


In [358]:
map_food = folium.Map(location=[ bor['long'][0], bor['lati'][0]], zoom_start=8)

# add markers to map
for lat, lng, label in zip(df_det['latitude'],
                           df_det['longitude'],
                           df_det['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_food)  
    
map_food

In [361]:
toro = df_det[df_det.Borough.str.contains('Toronto',case=False)]
toro.columns

Index(['id', 'name', 'latitude', 'longitude', 'distance_from_location',
       'address', 'cuisine', 'Borough'],
      dtype='object')

In [363]:
# one hot encoding
toro_onehot = pd.get_dummies(toro[['cuisine']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toro_onehot['Neighborhood'] = toro['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot = toro_onehot[fixed_columns]

toro_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Café,Chinese Restaurant,Creperie,...,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Central Toronto,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [365]:
print(toro_onehot.shape)
toro_grouped = toro_onehot.groupby('Neighborhood').mean().reset_index()
toro_grouped

(156, 35)


,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Café,Chinese Restaurant,Creperie,...,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Central Toronto,0.06,0.04,0.04,0.06,0.00,0.02,0.12000000,0.04,0.00,...,0.00,0.02,0.02,0.08000000,0.02,0.06,0.04,0.04,0.04,0.02
1,Downtown Toronto,0.02,0.02,0.02,0.04,0.02,0.02,0.08000000,0.04,0.02,...,0.02,0.02,0.06,0.08000000,0.02,0.06,0.02,0.02,0.02,0.02
2,East Toronto,0.00,0.00,0.00,0.00,0.00,0.00,0.16666667,0.00,0.00,...,0.00,0.00,0.00,0.16666667,0.00,0.00,0.00,0.00,0.00,0.00
3,West Toronto,0.06,0.04,0.04,0.06,0.00,0.02,0.12000000,0.04,0.00,...,0.00,0.02,0.02,0.08000000,0.02,0.06,0.04,0.04,0.04,0.02


In [408]:
num_cuisine = 5

for neig in toro_grouped['Neighborhood']:
    print(neig)
    temp = toro_grouped[toro_grouped['Neighborhood'] == neig].T.reset_index()
    temp.columns = ['cuisine','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_cuisine))
    print('\n')

Central Toronto
               cuisine  freq
0                 Café  0.12
1  Japanese Restaurant  0.08
2           Restaurant  0.08
3   Seafood Restaurant  0.06
4  American Restaurant  0.06


Downtown Toronto
               cuisine  freq
0  Japanese Restaurant  0.08
1                 Café  0.08
2           Restaurant  0.08
3            Gastropub  0.06
4     Ramen Restaurant  0.06


East Toronto
              cuisine  freq
0          Food Court  0.33
1       Deli / Bodega  0.17
2  Falafel Restaurant  0.17
3                Café  0.17
4          Restaurant  0.17


West Toronto
               cuisine  freq
0                 Café  0.12
1  Japanese Restaurant  0.08
2           Restaurant  0.08
3   Seafood Restaurant  0.06
4  American Restaurant  0.06




In [370]:
#k-means clustring
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [396]:
#clustring neighbourhoods
kclusters = 2

toro_grouped_clustering = toro_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:4] 

array([0, 0, 1, 0], dtype=int32)

In [377]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [409]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Cuisine'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Cuisine'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toro_grouped['Neighborhood']

for ind in np.arange(toro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Cuisine,2nd Most Common Cuisine,3rd Most Common Cuisine,4th Most Common Cuisine,5th Most Common Cuisine
0,Central Toronto,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
1,Downtown Toronto,Japanese Restaurant,Restaurant,Café,Seafood Restaurant,Ramen Restaurant
2,East Toronto,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
3,West Toronto,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot


In [398]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toro_merged = toro[['Borough','latitude','longitude']]
toro_merged.rename(columns={'Borough':'Neighborhood'},inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toro_merged = toro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toro_merged.head()

/home/sid/.local/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.65314384,-79.38198017,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
1,Central Toronto,43.65526772,-79.38516507,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
2,Central Toronto,43.65453649,-79.38088886,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
3,Central Toronto,43.65489528,-79.38505238,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
4,Central Toronto,43.65540380,-79.38192890,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot


In [392]:
toro_merged.head()

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.65314384,-79.38198017,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
1,Central Toronto,43.65526772,-79.38516507,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
2,Central Toronto,43.65453649,-79.38088886,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
3,Central Toronto,43.65489528,-79.38505238,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
4,Central Toronto,43.65540380,-79.38192890,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot


In [399]:
map_clusters = folium.Map(location=[bor['lati'][0], bor['long'][0]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toro_merged['latitude'], toro_merged['longitude'], toro_merged['Neighborhood'], toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [404]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0]#, toro_merged.columns[[0] + list(range(5, toro_merged.shape[1]))]]

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,43.65314384,-79.38198017,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
1,Central Toronto,43.65526772,-79.38516507,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
2,Central Toronto,43.65453649,-79.38088886,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
3,Central Toronto,43.65489528,-79.38505238,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
4,Central Toronto,43.65540380,-79.38192890,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
...,...,...,...,...,...,...,...,...,...
273,West Toronto,43.65839479,-79.38533766,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
274,West Toronto,43.64745207,-79.38132002,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
275,West Toronto,43.64883150,-79.38197006,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot
276,West Toronto,43.65022782,-79.38486260,0,Café,Japanese Restaurant,Restaurant,American Restaurant,Breakfast Spot


In [407]:
toro_merged.loc[toro_merged['Cluster Labels'] == 1]

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
100,East Toronto,43.72181380,-79.37652043,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
101,East Toronto,43.72162952,-79.37642455,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
102,East Toronto,43.72161664,-79.37592540,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
103,East Toronto,43.72178602,-79.37415044,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
104,East Toronto,43.72045203,-79.37793909,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
105,East Toronto,43.72731127,-79.37783462,1,Food Court,Café,Falafel Restaurant,Restaurant,Deli / Bodega
